## standard imports

In [251]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import requests
import json
import re
import pprint
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
from bs4 import BeautifulSoup
#from termcolor import colored

## setting up sqlite

In [252]:
database = "pitchfork.sqlite"
conn = sql.connect(database)
c = conn.cursor()

In [258]:
band_query = """
SELECT 
    r.artist,
    y.year,
    COUNT(*) counter
FROM reviews r
INNER JOIN years y ON y.reviewid == r.reviewid
GROUP BY r.artist
"""
df_bands = pd.read_sql_query(band_query, conn)

In [259]:
bands = df_bands.artist
bands[150:160]

150               africa hitech
151               africaine 808
152         african head charge
153           african virtuoses
154    africans with mainframes
155            afrika bambaataa
156                   afrirampo
157                  afterhours
158                         afx
159                 against me!
Name: artist, dtype: object

## We're trying three different approaches
1. Using genders from Wikidata
2. Using Categories from Wikipedia with the wiki API
3. Counting Pronouns

## 1. Approach: Using Wikidata


In [261]:
def find_gender_data(url, http):
    """
    given the unique wikidata URL of a profile,
    this function returns the gender if it is given
    """
    gender = "unknown"
    response = http.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    spoon = soup.find('div', id= "P21")
    if spoon != None:
        gender = spoon.find("div", class_="wikibase-snakview-value wikibase-snakview-variation-valuesnak").get_text()     
    return gender

def find_page_id_data(base_url, artist, http, pattern=r'(?<=#58;)(.*)(?=&#124)'):
    """
    given an artist name,
    this function return the unique wikidata URL to the artist.
    To avoid dissambiguation we add the suffixes when necessary...
    """
    variations = ['_(band)', '_(musician)', '_(singer)', '_(rapper)', '']
    for variation in variations:
        url = base_url + artist + variation
        
        response = http.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        spoon = soup.find_all('th', {'id': re.compile(r'&#58')})
        
        if len(spoon) > 0:
            spoon = spoon[0].get("id")
            m = re.search(pattern, spoon)
            link = "https:" + m.group(0)
            #print(colored(f'found with suffix {variation}', 'green'))
            return link 
            
    #print(colored(f'nothing found for {band}', 'red'))
    return  None


def get_wikidata(artist_list, verbose=True):
    """
    the wikidata approach to finding the data.
    We use the unique identifier wikidata pages and scrape them with bs4 to get
    info about the artists gender
    the function takes a list of artists and returns a dictionary with the 
    artists and their respective genders"""
    
    time_before = time.time()
    
    #### setup    
    artist_list = artist_list.str.title().str.strip().str.replace(" ", "_").str.replace("&", "%26")
    base_url = "https://en.wikipedia.org/wiki/"
    pattern = r'(?<=#58;)(.*)(?=&#124)'
    band_dict = {}
    time_before = time.time()
        
    #### retry strategy
    retry_strategy = Retry(
        total=3,
        status_forcelist=[429, 500, 502, 503, 504],
        method_whitelist=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)

    
    for counter, artist in enumerate(artist_list): 
        
        #### find the wikidata page link
        link = find_page_id_data(base_url, artist, http)
        gender = "unknown"

        #### tells us how we're doing with the algorithm
        if verbose:
            print (f"working on band number {counter} of {len(artist_list)}", end="\r")
        
        #### results
        if link != None:
            gender = find_gender_data(link, http) 
        band_dict[artist] = gender
        
        #print(artist, gender) 
        
        
    #### timer for final print
    elapsed_time = time.time() - time_before
    print(f"this approach took {elapsed_time} seconds for a total of {len(artist_list)} bands")
    
    return band_dict

In [262]:
band_dict_data = get_wikidata(bands[2000:2500], verbose=True)

this approach took 1322.7043149471283 seconds for a total of 500 bands


In [263]:
df_data = pd.DataFrame(pd.Series(band_dict_data).values, pd.Series(band_dict_data).index)
df_data.rename(columns={0: "gender_data"}, inplace=True)
df_data.index.name = "artist"

## 2. Approach: Use Categories via Wiki API

### Some notes on the API
the &redirects links directly to the final redirect. We then use the page_id of the final redirect to do our second query. (like in the last line in the next cell) This is important to get the right results...

In [260]:
# a standard API request
r = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&prop=categories&redirects&titles=Ben_Harper")
r.json()

## A band without redirect.
r = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&titles=Acid_Mothers_Temple_%26_The_Cosmic_Inferno")
r.json()

## A band with redirect. Note how it gives us a new pageid. The one of the final redirect...
r = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&titles=Acid_Mothers_Temple_%26_The_Cosmic_Inferno&redirects")
r.json()

## A band with redirect that is already the final landing page. The query yields essentially the same results
r = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&titles=Acid_Mothers_Temple&redirects")
r.json()

## A band with redirect and direct query for categories. The golden ticket...
r = requests.get("https://en.wikipedia.org/w/api.php?action=query&format=json&prop=categories&redirects&titles=Acid_Mothers_Temple_%26_The_Cosmic_Inferno")
_=r.json()


In [264]:
pp = pprint.PrettyPrinter()

def scrape_categories(band_list, redirect=True, verbose=True):
    """
    given a list of bands, this function returns all the categories the bands belong to on Wikipedia.
    The function does some preprocessing of the band names such as replacing spaces with "_" and 
    making the first letter of a word a capital and a few more operations...
    
    the function return a dict with the categories and a list of bands that have not been found
    
    redirect is a very important hypervariable. 
    It makes sure we look at the final redirect of the url we want to query. 
    We can turn it off, potentially speeding up the algorithm but that would yield inferior results...
    
    verbose just tells us where we're at in our process...
    """
    ### string preprocessing
    band_list = band_list.str.title().str.strip().str.replace(" ", "_").str.replace("&", "%26")
    
    #### setup
    not_found = []
    band_dict = {}
    time_before = time.time()
    base_url = 'https://en.wikipedia.org/w/api.php?action=query&format=json&prop=categories'
    
    if redirect:
        base_url += "&redirects"
        
    #### retry strategy
    retry_strategy = Retry(
        total=3,
        status_forcelist=[429, 500, 502, 503, 504],
        method_whitelist=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)
    
    for counter,band in enumerate(band_list):
        
        ##### speakerbox
        if verbose:
            print (f"working on band number {counter} of {len(band_list)}", end="\r")
            
        #### setup. We set up a helperfunction to return the page_id and the parsed query from an artist         
        page_id, url, parsed, suffix = find_page_id_cat(base_url, band, http)

        #### page_id -1 indicates a non-existant page. all other cases will be stripped in else
        if page_id == "-1":
            not_found.append(band)  
            
        else:
            category_path = parsed.get("query").get("pages").get(page_id).get("categories")
            category_list = []
            for categories in category_path:
                category_list.append(categories.get("title").replace("Category:", ""))
  
            continuation = parsed.get("continue")           
            while continuation != None: #### The api only gives us 10 results. for more we need to select &continue
                next_item = continuation.get("clcontinue")
                new_url = url + f'&continue={continuation.get("continue")}&clcontinue={next_item}'
                parsed = json.loads(http.get(new_url).content)
                category_path = parsed.get("query").get("pages").get(page_id).get("categories")
                for categories in category_path:
                    category_list.append(categories.get("title").replace("Category:", ""))   
                continuation = parsed.get("continue") 
            category_list.append(suffix)    
            band_dict[band] = category_list 
            
    #### timer for final print
    elapsed_time = time.time() - time_before
    print(f"this approach took {elapsed_time} seconds for a total of {len(band_list)} bands")
    return band_dict, not_found


def find_page_id_cat(base_url, band, http):
    """
    a helper function that finds the page id of a given wikipedia article. 
    Additionally it adds suffixes to avoid ambivalent names or unwanted redirects such as 'Air', or 'Foals' for example
    """
    
    variations = ['_(band)', '_(musician)', '_(singer)', ''] #could include _(rapper) but it's rare and would slow down the algo
    for variation in variations:
        url = base_url + f'&titles={band}{variation}'
        response = http.get(url)
        parsed = json.loads(response.content)
        page_id = [*parsed.get("query").get("pages")][0]
        if page_id != "-1":
            return page_id, url, parsed, variation
    return page_id, url, None, variation

In [265]:
band_dict_cat, not_found = scrape_categories(bands[2000:2500])
print("Here are the results of the algorithm with redirect:\n\n")
print(f"it hasn't found the following {len(not_found)} bands: \n", not_found, "\n")
print(f"it has found categories for {len(band_dict_cat)} bands. Here they are...\n")
pp.pprint(band_dict_cat)

this approach took 472.6943669319153 seconds for a total of 500 bands
Here are the results of the algorithm with redirect:


it hasn't found the following 167 bands: 
 ['Dj_Nate', 'Dj_Nature', 'Dj_Paypal', 'Dj_Q', 'Dj_Quik', 'Dj_Quik,_Kurupt', 'Dj_Quik,_Problem', 'Dj_Rashad', 'Dj_Richard', 'Dj_Roc', 'Dj_Rude_One', 'Dj_Shadow', 'Dj_Signify', 'Dj_Snake', 'Dj_Sotofett', 'Dj_Spinn', 'Dj_Spinna', 'Dj_Spoko', 'Dj_Spooky,_Killah_Priest,_Ward_29', 'Dj_Spooky,_Scanner', 'Dj_Sprinkles', 'Dj_T.', 'Dj_Target,_Riko', 'Dj_Zeph', 'Djds', 'Dlek', 'Dm_Stith', 'Dm-Funk', 'Dm-Funk,_Steve_Arrington', 'Dodsferd', 'Dog_Shredder', 'Doley_Bernays', 'Dolphins_Into_The_Future', 'Dominik_Eulberg', 'Domo_Genesis,_The_Alchemist', 'Don_Trip,_Starlito', 'Donato_Dozzy', 'Donato_Dozzy,_Nuel', 'Donmonique', 'Donnacha_Costello', 'Donnie_%26_Joe_Emerson', 'Donnie_Trumpet_%26_The_Social_Experiment', 'Dorine_Muraille', 'Doris_Henson', 'Double_Leopards', 'Doubled_Yellow_Swans', 'Doug_Hilsinger,_Caroleen_Beatty', 'Doug_Hream

                    'American film score composers',
                    'American jazz composers',
                    'American jazz musicians',
                    'American jazz pianists',
                    'American male conductors (music)',
                    'American male pianists',
                    'Articles containing video clips',
                    'Articles needing additional references from January 2018',
                    'Articles with Encyclopædia Britannica links',
                    'Articles with IBDb links',
                    'Articles with hCards',
                    'Articles with short description',
                    'Articles with unsourced statements from April 2016',
                    'Bethlehem Records artists',
                    'Big band bandleaders',
                    'Big band pianists',
                    'Black Lion Records artists',
                    'Black conductors',
                    'Broadway composers and lyricists',
  

                   'Pentecostals from Tennessee',
                   'People from Memphis, Tennessee',
                   'People from Tupelo, Mississippi',
                   'Presidential Medal of Freedom recipients',
                   'RCA Victor artists',
                   'Rock and roll musicians',
                   'Short description matches Wikidata',
                   'Singers from Mississippi',
                   'Singers from Tennessee',
                   'Southern gospel performers',
                   'Sun Records artists',
                   'Tank personnel',
                   'Traditional pop music singers',
                   'Turner Classic Movies person ID same as Wikidata',
                   'Twin people from the United States',
                   'United States Army soldiers',
                   'Use American English from June 2020',
                   'Use mdy dates from June 2020',
                   'Wikipedia articles with BIBSYS identifiers',
            

                'Political music',
                'Webarchive template wayback links',
                'Wikipedia articles with BNF identifiers',
                'Wikipedia articles with GND identifiers',
                'Wikipedia articles with ISNI identifiers',
                'Wikipedia articles with LCCN identifiers',
                'Wikipedia articles with MusicBrainz identifiers',
                'Wikipedia articles with VIAF identifiers',
                'Wikipedia articles with WorldCat identifiers',
                ''],
 'Eyeless_In_Gaza': ['All article disambiguation pages',
                     'All disambiguation pages',
                     'Disambiguation pages',
                     'Disambiguation pages with short descriptions',
                     ''],
 'Eyes_Adrift': ['2002 establishments in the United States',
                 'Album articles lacking alt text for covers',
                 'American alternative country groups',
                 'American alternati

In [273]:
df_cat = pd.DataFrame(pd.Series(band_dict_cat).values, pd.Series(band_dict_cat).index)
df_cat.rename(columns={0: "categories"}, inplace=True)
df_cat.loc[:, 'suffix'] = df_cat.categories.map(lambda x: x[-1])
df_cat.loc[:, 'categories'] = df_cat.categories.map(lambda x: x[:-1])
df_cat.index.name = "artist"
df_cat

,categories,suffix
artist,,
Dj_Spooky,"[1970 births, All BLP articles lacking sources...",
Django_Django,"[Articles with hCards, Because Music artists, ...",
Dna,"[Articles with Curlie links, Articles with hAu...",
Dntel,"[All BLP articles lacking sources, American el...",
Do_Make_Say_Think,"[1995 establishments in Ontario, All articles ...",
...,...,...
Faun_Fables,"[1999 establishments in California, All articl...",
Faunts,"[2000 establishments in Alberta, All articles ...",
Faust,"[All articles needing additional references, A...",_(band)


In [274]:
## just backing up
backup_cat=df_cat.copy()
backup_data = df_data.copy()

## Concatenating the two dataframes to find out which method is better

In [316]:
df = pd.concat([df_cat, df_data], axis=1, sort=False)
df

,categories,suffix,gender_data
Dj_Spooky,"[1970 births, All BLP articles lacking sources...",,male
Django_Django,"[Articles with hCards, Because Music artists, ...",,unknown
Dna,"[Articles with Curlie links, Articles with hAu...",,unknown
Dntel,"[All BLP articles lacking sources, American el...",,male
Do_Make_Say_Think,"[1995 establishments in Ontario, All articles ...",,unknown
...,...,...,...
Father_Yod_And_The_Source_Family,NaN,NaN,unknown
Father'S_Children,NaN,NaN,unknown
"Faust,_Dlek",NaN,NaN,unknown
Fc/Kahuna,NaN,NaN,unknown


In [317]:
## replace the nan values to make the next step work

df.categories = df.categories.fillna("not found")
df.suffix = df.suffix.fillna("")

In [318]:
def find_gender_cat(df, band_dict):
    df["gender_cat"] = "unknown"
    for band in band_dict:
        for cat in df.loc[band,"categories"]:
            if (("male" in cat) or ("Female" in cat) or ("Boy" in cat) or ("boy" in cat)):
                df.loc[band,"gender_cat"] = "male"
            if (("Female" in cat) or ("female" in cat) or ("girl" in cat) or ("Girl" in cat)):
                df.loc[band,"gender_cat"] = "female"
            if ("Transgender" in cat or "transsexual"in cat):
                df.loc[band,"gender_cat"] = "other"
                
find_gender_cat(df, list(df.index))
df

,categories,suffix,gender_data,gender_cat
Dj_Spooky,"[1970 births, All BLP articles lacking sources...",,male,unknown
Django_Django,"[Articles with hCards, Because Music artists, ...",,unknown,unknown
Dna,"[Articles with Curlie links, Articles with hAu...",,unknown,unknown
Dntel,"[All BLP articles lacking sources, American el...",,male,unknown
Do_Make_Say_Think,"[1995 establishments in Ontario, All articles ...",,unknown,unknown
...,...,...,...,...
Father_Yod_And_The_Source_Family,not found,,unknown,unknown
Father'S_Children,not found,,unknown,unknown
"Faust,_Dlek",not found,,unknown,unknown
Fc/Kahuna,not found,,unknown,unknown


In [319]:
#where do cat and data gender not lign up?

df[df["gender_cat"] != df["gender_data"]]

,categories,suffix,gender_data,gender_cat
Dj_Spooky,"[1970 births, All BLP articles lacking sources...",,male,unknown
Dntel,"[All BLP articles lacking sources, American el...",,male,unknown
Doctor_Rockit,"[1972 births, All BLP articles lacking sources...",,male,unknown
Dosh,"[1972 births, American electronic musicians, A...",_(musician),male,unknown
Doveman,"[1981 births, 21st-century American keyboardis...",,male,unknown
Dr._Octagon,"[Fictional extraterrestrial characters, Fictio...",,male,unknown
Drop_The_Lime,"[American electronic musicians, American peopl...",,male,unknown
Du_Blonde,"[1990 births, 21st-century British singers, Ar...",,non-binary,other
Dudley_Perkins,"[All article disambiguation pages, All disambi...",,male,unknown
Dum_Dum_Girls,"[All-female bands, All articles with unsourced...",,unknown,female


In [ ]:
#df.to_csv('some list.csv') ### To save

#df = pd.read_csv('some list.csv') ### to load
#df.rename(columns={"Unnamed: 0": "artist"}, inplace=True)
#df.set_index("artist")
#df["categories"] = df.categories.apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(", "))

## We got some wrong results. 
for example articles linking to disambiguation pages.

## Returns all bands / artists with disambiguation pages
This means we have been linked to the wrong page. However, since (almost) none of them had a gender assigned to them it doesn't seem to be a huge problem right now. We might look into the root of the problem at some later time

In [320]:
selection = ["All disambiguation pages"]
mask = df.categories.apply(lambda x: np.intersect1d(x, selection).size > 0)
disamb = df[mask]
disamb

,categories,suffix,gender_data,gender_cat
Do_Or_Die,"[All article disambiguation pages, All disambi...",,unknown,unknown
Doom,"[All article disambiguation pages, All disambi...",_(band),unknown,unknown
Doss,"[All article disambiguation pages, All disambi...",,unknown,unknown
Dual_Identity,"[All article disambiguation pages, All disambi...",,unknown,unknown
Dudley_Perkins,"[All article disambiguation pages, All disambi...",,male,unknown
Dusted,"[All article disambiguation pages, All disambi...",_(band),unknown,unknown
Earth,"[All article disambiguation pages, All disambi...",_(band),unknown,unknown
Eaters,"[All article disambiguation pages, All disambi...",,unknown,unknown
Ema,"[All article disambiguation pages, All disambi...",,unknown,unknown
Embrace,"[All article disambiguation pages, All disambi...",_(band),unknown,unknown


## 3. Approach: Using the pronouns

In [321]:
#band_list = band_list.str.title().str.strip().str.replace(" ", "_").str.replace("&", "%26")
def pronoun_gender(df, band_list, verbose=True, limit=2):
    
    time_before = time.time()
    new_df = df.copy()
    new_df["gender_pronoun"] = "unknown"
    male_identifiers = r'\bhe\b|\bhis\b'
    male_expr = re.compile(male_identifiers, re.IGNORECASE)

    female_identifiers = r'\bshe\b|\bher\b'
    female_expr = re.compile(female_identifiers, re.IGNORECASE)

    base_url = "https://en.wikipedia.org/wiki/"
    
    for counter, band in enumerate(band_list.str.title().str.strip().str.replace(" ", "_").str.replace("&", "%26")):              
        if df.loc[band]["suffix"] != "_(band)": 
            
            pronoun = "unknown"
            
            if verbose:
                print (f"working on band number {counter} of {len(band_list)}", end="\r")  
                
            url = base_url + f"{band}" + df.loc[band]["suffix"]

            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            spoon = soup.find(id="mw-content-text").find_all("p", limit=limit)

            male = male_expr.findall(str(spoon))
            female = female_expr.findall(str(spoon))

            if len(male) > len (female):
                pronoun = "male" 
            elif len(male) < len (female):
                pronoun = "female"
            new_df.loc[band]["gender_pronoun"] = pronoun
    elapsed_time = time.time() - time_before
    print(f"this approach took {elapsed_time} seconds for a total of {len(band_list)} bands")
            
    return new_df

In [322]:
df = pronoun_gender(df, bands[2000:2500], limit=2, verbose=True)
#df_after.gender.value_counts()

this approach took 166.46960616111755 seconds for a total of 500 bands


In [323]:
## Where gender_data  and gender_pronoun are not the same. Quite a big list...(over10%)
df[(df["gender_data"] == "unknown" ) & ( df["gender_pronoun"] != "unknown" )]

,categories,suffix,gender_data,gender_cat,gender_pronoun
Dolorean,"[1999 establishments in Oregon, Articles with ...",,unknown,unknown,male
Dope_Body,"[American noise rock music groups, Articles wi...",,unknown,unknown,male
Doughboyz_Cashout,"[American record labels, Companies based in At...",,unknown,unknown,male
Drive_Like_Jehu,"[American emo musical groups, American post-ha...",,unknown,unknown,male
Drowners,[Alternative rock groups from New York (state)...,,unknown,unknown,male
Dum_Dum_Girls,"[All-female bands, All articles with unsourced...",,unknown,female,female
Dungen,"[1999 establishments in Sweden, All articles n...",,unknown,unknown,male
Eagles_Of_Death_Metal,"[1998 establishments in California, All articl...",,unknown,unknown,male
Eartheater,"[1989 births, 21st-century American women musi...",_(musician),unknown,unknown,female
Edward_Sharpe_%26_The_Magnetic_Zeros,"[2005 establishments in California, All articl...",,unknown,unknown,male


In [324]:
df.to_csv('500 samples.csv') ### Saving the dataset